In [ ]:
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
df=pd.read_csv("/content/advertising.csv")
df.head()

,TV,Radio,Newspaper,Sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,12.0
3,151.5,41.3,58.5,16.5
4,180.8,10.8,58.4,17.9


In [ ]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TV         200 non-null    float64
 1   Radio      200 non-null    float64
 2   Newspaper  200 non-null    float64
 3   Sales      200 non-null    float64
dtypes: float64(4)
memory usage: 6.4 KB


,TV,Radio,Newspaper,Sales
count,200.000000,200.000000,200.000000,200.000000
mean,147.042500,23.264000,30.554000,15.130500
std,85.854236,14.846809,21.778621,5.283892
min,0.700000,0.000000,0.300000,1.600000
25%,74.375000,9.975000,12.750000,11.000000
50%,149.750000,22.900000,25.750000,16.000000
75%,218.825000,36.525000,45.100000,19.050000
max,296.400000,49.600000,114.000000,27.000000


In [ ]:
X = df.drop('Sales', axis=1)
y = df['Sales']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.05, 0.1, 0.2]
}

# Create the Gradient Boosting Regression model
model = GradientBoostingRegressor()

# Perform Grid Search to find the best hyperparameters
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Make predictions on the testing set
y_pred = best_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Best Model Parameters: {grid_search.best_params_}')
print(f'Mean Squared Error: {mse}')

Best Model Parameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 50}
Mean Squared Error: 1.8911608582655794


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

sales_stats = df['Sales'].describe()

# Determine threshold values based on percentiles
low_sales_threshold = sales_stats['25%']  # Lower quartile (25th percentile)
high_sales_threshold = sales_stats['75%']  # Upper quartile (75th percentile)

print(f'Low Sales Threshold: {low_sales_threshold}')
print(f'High Sales Threshold: {high_sales_threshold}')

np.random.seed(42)
noise = np.random.normal(loc=0, scale=0.01, size=len(y_test))
y_test_noisy = y_test + noise


y_pred_class_noisy = []
for pred in y_pred:
    if pred < low_sales_threshold:
        y_pred_class_noisy.append('Low')
    elif pred < high_sales_threshold:
        y_pred_class_noisy.append('Medium')
    else:
        y_pred_class_noisy.append('High')


y_test_class_noisy = []
for true_label in y_test_noisy:
    if true_label < low_sales_threshold:
        y_test_class_noisy.append('Low')
    elif true_label < high_sales_threshold:
        y_test_class_noisy.append('Medium')
    else:
        y_test_class_noisy.append('High')

accuracy_noisy = sum(1 for pred, true_label in zip(y_pred_class_noisy, y_test_class_noisy) if pred == true_label) / len(y_test_class_noisy)

print(f'Accuracy with noise: {accuracy_noisy}')


conf_matrix = confusion_matrix(y_test_class_noisy, y_pred_class_noisy)
print("Confusion Matrix:")
print(conf_matrix)


class_report = classification_report(y_test_class_noisy, y_pred_class_noisy)
print("Classification Report:")
print(class_report)


Low Sales Threshold: 11.0
High Sales Threshold: 19.05
Accuracy with noise: 0.875
Confusion Matrix:
[[10  0  0]
 [ 0 10  3]
 [ 1  1 15]]
Classification Report:
              precision    recall  f1-score   support

        High       0.91      1.00      0.95        10
         Low       0.91      0.77      0.83        13
      Medium       0.83      0.88      0.86        17

    accuracy                           0.88        40
   macro avg       0.88      0.88      0.88        40
weighted avg       0.88      0.88      0.87        40

